1) raw dataset
2) keywords based extraction
3) train/test split
4) sample a specific number of samples (high cardinality)
5) paste the instruction

In [1]:
from huggingface_hub import logout, notebook_login
logout()
notebook_login()

In [17]:
## dataset load

from datasets import load_dataset

# qa datasets
medqa = load_dataset("GBaker/MedQA-USMLE-4-options")
# medqa = load_dataset("bigbio/med_qa")
medmcqa = load_dataset("openlifescienceai/medmcqa")
# pubmedqa = load_dataset("bigbio/pubmed_qa")
# pubmedqa = load_dataset("qiaojin/PubMedQA", "pqa_artificial")
# bioasq = load_dataset("rag-datasets/rag-mini-bioasq", "question-answer-passages")
# mednli = load_dataset("bigbio/mednli")

# nli datasets
# mednli = load_dataset("cnut1648/mnli_resampled_as_mednli")
bionli = load_dataset("clinicalnlplab/BioNLI_test")

# information extraction datasets
# cie_mse = load_dataset("mitclinicalml/clinical-ie", "medication_status", trust_remote_code=True) # medication status extraction
# cie_ccr = load_dataset("mitclinicalml/clinical-ie", "coreference", trust_remote_code=True) # clinical coreference resolution

# medmentions = load_dataset("bigbio/medmentions", trust_remote_code=True)
# bc5cdr = load_dataset("bigbio/bc5cdr", trust_remote_code=True)

# chemdisgene = load_dataset("bigbio/chem_dis_gene", trust_remote_code=True)
# biored = load_dataset("bigbio/biored", trust_remote_code=True)
# biorelex = load_dataset("bigbio/biorelex", trust_remote_code=True)
biorel = load_dataset("DFKI-SLT/BioRel")
# biorex = load_dataset("bigbio/biorelex", trust_remote_code=True)

# generation task about clinical skills datasets
# medichat = load_dataset("Mostafijur/medichat_conversation", "medichat_subset1") # ~ medichat_subset15 # conv2note
icliniq = load_dataset("lavita/ChatDoctor-iCliniq")
# mediqa = load_dataset("jonathankang/EN-MEDIQA")
pubmed = load_dataset("ccdv/pubmed-summarization")

In [18]:
icliniq = icliniq['train'].train_test_split(test_size=0.3, seed=42)
# pubmedqa = pubmedqa['train'].train_test_split(test_size=0.3, seed=42)

In [19]:
print("medqa")
print(medqa)
print('------------------------------------------------------------------------------'* 2) 
print("medmcqa")
print(medmcqa)
print('------------------------------------------------------------------------------'* 2) 
# print("pubmedqa")
# print(pubmedqa)
# print('------------------------------------------------------------------------------'* 2) 
# print("mednli")
# print(mednli)
# print('------------------------------------------------------------------------------'* 2) 
print("bionli")
print(bionli)
print('------------------------------------------------------------------------------'* 2) 
# print("cie_mse")
# print(cie_mse)
# print('------------------------------------------------------------------------------'* 2) 
# print("cie_ccr")
# print(cie_ccr)
# print('------------------------------------------------------------------------------'* 2) 
# print("medmentions")
# print(medmentions)
# print('------------------------------------------------------------------------------'* 2) 
# print("bc5cdr")
# print(bc5cdr)
# print('------------------------------------------------------------------------------'* 2) 
# print("chem-dis-gene")
# print(chemdisgene)
# print('------------------------------------------------------------------------------'* 2) 
# print("biored")
# print(biored)
# print('------------------------------------------------------------------------------'* 2) 
# print("biorelex")
# print(biorelex)
# print('------------------------------------------------------------------------------'* 2) 
print("biorel")
print(biorel)
print('------------------------------------------------------------------------------'* 2) 
# print("biorex")
# print(biorex)
# print('------------------------------------------------------------------------------'* 2) 
# print("medichat")
# print(medichat)
# print('------------------------------------------------------------------------------'* 2) 
print("icliniq")
print(icliniq)
print('------------------------------------------------------------------------------'* 2) 
# print("mediqa")
# print(mediqa)
# print('------------------------------------------------------------------------------'* 2) 
print("pubmed")
print(pubmed)

medqa
DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 1273
    })
})
------------------------------------------------------------------------------------------------------------------------------------------------------------
medmcqa
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_na

In [ ]:
import pandas as pd
import os

 
keywords = [
    # 질환 관련
    "diabetes", "HbA1c", "blood sugar", 
    "glucose", "ketoacidosis",
    
    # 약물 및 치료
    "insuline",
    "metformin", "SGLT2 inhibitors", "GLP-1 receptor agonist",

    # 합병증
    "neuropathy", "retinopathy", "nephropathy",
]

def contains_keywords(text, keywords):
    if not text or not isinstance(text, str): 
        return False
    return any(keyword in text.lower() for keyword in keywords)

def process_dataset_split(dataset_name, split_name, dataset, text_columns, output_file, keywords=["diabetes", "insulin"]):
    rows = []
    for i in range(len(dataset)):
        row = dataset[i]

        for col in text_columns:
            if col in row:
                text = row[col]

            elif col == "abstract" and "passages" in row:
                if isinstance(row["passages"], list):
                    text = " ".join([p["text"] for p in row["passages"] if p.get("type") == "abstract"])
                else:
                    text = ""
            else:
                text = ""

            if not text:
                continue

            is_related = 1 if contains_keywords(text, keywords) else 0

            rows.append({
                "dataset": dataset_name,
                "split_data": split_name,
                "features": row,  
                "input": text,
                "output": is_related
            })

    if rows:  
        df = pd.DataFrame(rows)
        df.to_csv(output_file, mode="a", header=not os.path.exists(output_file), index=False)
        print(f"{dataset_name} ({split_name}): {len(rows)} rows processed and appended to {output_file}")
    else:
        print(f"{dataset_name} ({split_name}): No rows processed.")

datasets = {
    "medqa": {"splits": medqa, "columns": ["question"]},
    "medmcqa": {"splits": medmcqa, "columns": ["question"]},
    # "pubmedqa": {"splits": pubmedqa, "columns": ["question"]},
    # "mednli": {"splits": mednli, "columns": ["premise", "hypothesis"]},
    "bionli": {"splits": bionli, "columns": ["query", "answer"]},
    # "cie_mse": {"splits": cie_mse, "columns": ["snippet"]},
    # "cie_ccr": {"splits": cie_ccr, "columns": ["snippet"]},
    # "medmentions: {"splits": medmentions, "columns": ["passages"]},
    # "bc5cdr": {"splits": bc5cdr, "columns": ["passages"]},
    # "chemdisgene": {"splits": chemdisgene, "columns": ["passages"]},
    # "biored": {"splits": biored, "columns": ["passages"]},
    # "biorelex": {"splits": biorelex, "columns": ["text"]},
    "biorel": {"splits": biorel, "columns": ["text"]},
    # "biorex": {"splits": biorex, "columns": ["text"]},
    # "medichat": {"splits": medichat, "columns": ["conversation"]},
    "icliniq": {"splits": icliniq, "columns": ["input", "answer_icliniq"]},
    "pubmed": {"splits": pubmed, "columns": ["abstract"]},
}

output_file = os.path.join("all_data_filtered_by_keywords.csv")

for dataset_name, details in datasets.items():
    splits = details["splits"]
    text_columns = details["columns"] 

    for split_name, split_data in splits.items():
        process_dataset_split(dataset_name, split_name, split_data, text_columns, output_file, keywords=keywords)


medqa (train): 10178 rows processed and appended to all_data_filtered_by_keywords.csv
medqa (test): 1273 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (train): 182822 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (test): 6150 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (validation): 4183 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (train): 11088 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (validation): 25612 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (test): 12616 rows processed and appended to all_data_filtered_by_keywords.csv
biorel (train): 534277 rows processed and appended to all_data_filtered_by_keywords.csv
biorel (validation): 114506 rows processed and appended to all_data_filtered_by_keywords.csv
biorel (test): 114565 rows processed and appended to all_data_filtered_by_keywords.csv
icliniq (train): 10248 rows pr

In [ ]:
import pandas as pd

all_df = pd.read_csv("src/data/data1_diabetes/all_data_filtered_by_keywords.csv")

In [24]:
# data_filtered_by_keyword
diabetes = all_df[all_df['output'] == 1]

In [25]:
diabetes.head(2)

,dataset,split_data,features,input,output
15,medqa,train,{'question': 'A 68-year-old man presents to th...,A 68-year-old man presents to the emergency de...,1
17,medqa,train,{'question': 'A 68-year-old man comes to the p...,A 68-year-old man comes to the physician becau...,1


In [26]:
diabetes.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset  split_data
bionli   test            651
         train           615
         validation     1380
biorel   test           2603
         train         11644
         validation     2566
icliniq  test            162
         train           373
medmcqa  test             69
         train          1992
         validation       45
medqa    test            187
         train          1594
pubmed   test            544
         train          9119
         validation      537
Name: output, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

def re_train_test_split(dataframe, train_ratio=0.8):
    new_rows = []

    grouped = dataframe.groupby('dataset')

    for dataset_name, group in grouped:
        if dataset_name not in ['bionli', 'bioroel', 'icliniq', 'medmcqa', 'medqa', 'pubmed']: continue

        train_data, test_data = train_test_split(
            group,
            test_size=1-train_ratio,
            random_state=42,
        )

        train_data = train_data.copy()
        train_data['split_data'] = 'train'

        test_data = test_data.copy()
        test_data['split_data'] = 'test'

        new_rows.append(train_data)
        new_rows.append(test_data)

        result_df = pd.concat(new_rows, ignore_index=True)

    return result_df


diabetes2 = re_train_test_split(diabetes, train_ratio=0.8)

In [28]:
diabetes2.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset  split_data
bionli   test           530
         train         2116
icliniq  test           107
         train          428
medmcqa  test           422
         train         1684
medqa    test           357
         train         1424
pubmed   test          2040
         train         8160
Name: output, dtype: int64

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

sample_sizes = {
    "bionli": 2000,
    "biorel": 2000, 
    "icliniq": 500,
    "medmcqa": 1000,
    "medqa": 1000,
    # "pubmedqa": 2000,
    "pubmed": 2000,
}

model = SentenceTransformer('all-MiniLM-L6-v2')

final_combined_sample = None
highest_cardinality = -1


def sample_data(dataset, sample_sizes, split_data="train"):
    sampled_data = []
    for key, size in sample_sizes.items():
        data_subset = dataset[(dataset['dataset'] == key) & (dataset['split_data'] == split_data)]
        if len(data_subset) >= size:
            sampled_data.append(data_subset.sample(n=size, random_state=random.randint(1, 10000)))
        else:
            sampled_data.append(data_subset.sample(n=len(data_subset), random_state=random.randint(1, 10000)))
    return pd.concat(sampled_data)

def remove_duplicates(combined_sample, threshold=0.7):
    embeddings = model.encode(combined_sample['features'].tolist())
    similarity_matrix = cosine_similarity(embeddings)
    to_remove = set()

    for i in range(len(similarity_matrix)):
        for j in range(i + 1, len(similarity_matrix)):
            if similarity_matrix[i, j] > threshold:
                to_remove.add(j)
    
    return combined_sample.reset_index(drop=True).drop(index=list(to_remove))

from tqdm import tqdm

for iteration in tqdm(range(10)):

    combined_sample = sample_data(diabetes, sample_sizes)
    
    # cardinality 계산
    filtered_sample = remove_duplicates(combined_sample)
    current_cardinality = filtered_sample['features'].nunique()

    # cardinality가 더 높은 샘플 저장
    if current_cardinality > highest_cardinality:
        highest_cardinality = current_cardinality
        final_combined_train_sample = combined_sample

print(f"최종 선택된 샘플의 cardinality: {highest_cardinality}")
print(f"최종 샘플 크기: {len(final_combined_train_sample)}")


100%|██████████| 10/10 [10:09<00:00, 60.92s/it]

최종 선택된 샘플의 cardinality: 2505
최종 샘플 크기: 8988


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

sample_sizes = {
    "bionli": 500,
    "biorel": 500, 
    "icliniq": 125, 
    "medmcqa": 250,
    "medqa": 250,
    # "pubmedqa": 500,
    "pubmed": 500,
}

model = SentenceTransformer('all-MiniLM-L6-v2')

final_combined_sample = None
highest_cardinality = -1


def sample_data(dataset, sample_sizes, split_data="train"):
    sampled_data = []
    for key, size in sample_sizes.items():
        data_subset = dataset[(dataset['dataset'] == key) & (dataset['split_data'] == split_data)]
        if len(data_subset) >= size:
            sampled_data.append(data_subset.sample(n=size, random_state=random.randint(1, 10000)))
        else:
            sampled_data.append(data_subset.sample(n=len(data_subset), random_state=random.randint(1, 10000)))
    return pd.concat(sampled_data)


def remove_duplicates(combined_sample, threshold=0.7):
    embeddings = model.encode(combined_sample['features'].tolist())
    similarity_matrix = cosine_similarity(embeddings)
    to_remove = set()

    for i in range(len(similarity_matrix)):
        for j in range(i + 1, len(similarity_matrix)):
            if similarity_matrix[i, j] > threshold:
                to_remove.add(j)
    
    return combined_sample.reset_index(drop=True).drop(index=list(to_remove))

from tqdm import tqdm

for iteration in tqdm(range(10)):

    combined_sample = sample_data(diabetes, sample_sizes, split_data='test')
    
    # cardinality 계산
    filtered_sample = remove_duplicates(combined_sample)
    current_cardinality = filtered_sample['features'].nunique()

    # cardinality가 더 높은 샘플 저장
    if current_cardinality > highest_cardinality:
        highest_cardinality = current_cardinality
        final_combined_test_sample = combined_sample

print(f"최종 선택된 샘플의 cardinality: {highest_cardinality}")
print(f"최종 샘플 크기: {len(final_combined_test_sample)}")


100%|██████████| 10/10 [01:09<00:00,  6.96s/it]

최종 선택된 샘플의 cardinality: 843
최종 샘플 크기: 2381


In [ ]:
final_combined_train_sample.to_csv("final_combined_train_sample.csv")
final_combined_test_sample.to_csv("final_combined_test_sample.csv")

In [ ]:
import pandas as pd

final_combined_train_sample = pd.read_csv("src/data/data1_diabetes/final_combined_train_sample.csv")
final_combined_test_sample = pd.read_csv("/src/data/data1_diabetes/final_combined_test_sample.csv")

In [161]:
# train/test 겹치는 개수
len(pd.merge(final_combined_train_sample, final_combined_test_sample, on='features'))

0

In [162]:
import numpy as np

final_combined_train_sample['task'] = None

# bionli
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'bionli', 'task'] = 'nli' 

# biorel
biorel_indices = final_combined_train_sample[final_combined_train_sample['dataset'] == 'biorel'].index
biorel_splits = np.array_split(biorel_indices, 1)

biorel_tasks = [
    'ie_extract_relation',
    # 'ie_classify_relation',
    # 'ie_identify_relation_exist',
    # 'ie_generate_relation_pair'
]

for split, task in zip(biorel_splits, biorel_tasks):
    final_combined_train_sample.loc[split, 'task'] = task

# icliniq
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'icliniq', 'task'] = 'qa_subjective'

# medmcqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medmcqa', 'task'] = 'qa_objective'

# medqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medqa', 'task'] = 'qa_objective'

# pubmedqa
# pubmedqa_indices = final_combined_train_sample[final_combined_train_sample['dataset'] == 'pubmedqa'].index
# pubmedqa_splits = np.array_split(pubmedqa_indices, 4)

# pubmedqa_tasks = [
#     'qa_mesh_tagging',
#     'qa_short_answer',
#     'qa_long_answer',
#     'qa_complex_answer'
# ]

# for split, task in zip(pubmedqa_splits, pubmedqa_tasks):
#     final_combined_train_sample.loc[split, 'task'] = task

# pubmed
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'pubmed', 'task'] = 'summarization'

In [163]:
import numpy as np

final_combined_test_sample['task'] = None

# bionli
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'bionli', 'task'] = 'nli' 

# biorel
biorel_indices = final_combined_test_sample[final_combined_test_sample['dataset'] == 'biorel'].index
biorel_splits = np.array_split(biorel_indices, 1)

biorel_tasks = [
    'ie_extract_relation',
    # 'ie_classify_relation',
    # 'ie_identify_relation_exist',
    # 'ie_generate_relation_pair'
]

for split, task in zip(biorel_splits, biorel_tasks):
    final_combined_test_sample.loc[split, 'task'] = task

# icliniq
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'icliniq', 'task'] = 'qa_subjective'

# medmcqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medmcqa', 'task'] = 'qa_objective'

# medqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medqa', 'task'] = 'qa_objective'

# pubmedqa
# pubmedqa_indices = final_combined_test_sample[final_combined_test_sample['dataset'] == 'pubmedqa'].index
# pubmedqa_splits = np.array_split(pubmedqa_indices, 4)

# pubmedqa_tasks = [
#     'qa_mesh_tagging',
#     'qa_short_answer',
#     'qa_long_answer',
#     'qa_complex_answer'
# ]

# for split, task in zip(pubmedqa_splits, pubmedqa_tasks):
#     final_combined_test_sample.loc[split, 'task'] = task

# pubmed
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'pubmed', 'task'] = 'summarization'


In [ ]:
import random

instruction_mapping = {
    'nli': [
        "Given a premise and a hypothesis, determine their relationship: entailment, contradiction, or neutral.",
        "Read the given premise and hypothesis. Decide if the hypothesis logically follows from the premise.",
        "Classify the relationship between the premise and the hypothesis into one of three categories: entailment, contradiction, or neutral.",
        "Analyze the relationship between the given premise and hypothesis. Categorize it as entailment, contradiction, or neutral.",
        "Does the premise logically support the hypothesis? Answer as entailment, contradiction, or neutral.",
        "Evaluate if the hypothesis can be inferred from the premise. Label it as entailment, contradiction, or neutral.",
        "Does the hypothesis contradict the premise or is it entailed by it? If neither, classify it as neutral.",
        "For the given premise and hypothesis, determine their logical relationship: entailment, contradiction, or neutral."
    ],
    'ie_extract_relation': [
        "Extract the relationship described between the given entities in the text.",
        "Identify the type of relationship that connects the specified entities based on the provided text.",
        "Determine the specific relationship mentioned between the entities in the text.",
        "From the text, describe the relationship that exists between the provided entities."
    ],
    # 'ie_identify_relation_exist': [
    #     "Does the text describe any relationship between the specified entities? Answer Yes or No.",
    #     "Based on the text, determine if there is a connection or relationship between the given entities. Answer Yes or No.",
    #     "Identify whether a relationship exists between the specified entities in the text. Answer Yes or No.",
    #     "From the text, confirm whether there is a described relationship connecting the given entities. Answer Yes or No."
    # ],
    # 'ie_classify_relation': [
    #     "Classify the type of relationship described between the specified entities in the text.",
    #     "Identify the category that best describes the relationship between the entities based on the text.",
    #     "Determine whether the relationship described in the text falls into predefined categories such as physical, causal, or regulatory.",
    #     "Based on the text, categorize the relationship between the entities into its most relevant type."
    # ],
    # 'ie_generate_relation_pair': [
    #     "Generate all possible entity pairs from the text and identify their relationships.",
    #     "Extract all entity pairs mentioned in the text along with the relationships that connect them.",
    #     "Identify all entity pairs present in the text and specify their corresponding relationships if available.",
    #     "From the text, list all pairs of entities and describe the relationships that exist between them."
    # ],
    'qa_subjective': [
        "What are the next steps or follow-up actions suggested in the doctor’s response?",
        "Generate a comprehensive medical response based on the patient's query.",
        "Provide a concise and actionable answer to the patient's question.",
        "What advice or recommendations does the doctor provide in response to the patient's concerns?",
        "What follow-up steps does the doctor suggest in their response?"
    ],
    'qa_objective': [
        "Select the most appropriate answer for the given medical question from the provided options.",
        "Identify the correct answer to the medical question from the four given options.",
        "Pick the correct choice for the given medical question based on the provided options.",
        "Determine the correct option that best answers the given medical question.",
        "Choose the answer that accurately addresses the given medical query from the options provided.",
        "From the provided choices, select the option that correctly answers the medical question.",
        "Based on the medical question, select the most accurate answer from the given options.",
        "Identify the best answer for the given medical question from the options below.",
        "Select the correct answer that corresponds to the given medical question.",
        "Determine the appropriate choice for the medical question from the list of options."
    ],
    # 'qa_mesh_tagging': [
    #     "Identify all MeSH terms mentioned in the context and list them.",
    #     "Extract relevant MeSH terms from the provided text that describe the main topics.",
    #     "Tag the context with appropriate MeSH terms that represent its primary themes.",
    #     "Using the context, identify the MeSH terms that best summarize its content.",
    #     "Highlight the MeSH terms in the text that are crucial for understanding the main subject.",
    #     "Using the given context, tag the text with relevant MeSH terms that describe its focus.",
    #     "From the context, extract key MeSH terms that align with the primary topics discussed.",
    #     "Analyze the text and identify all MeSH terms that pertain to the discussed topics."
    # ],
    # 'qa_complex_answer': [
    #     "Predict the correct answer (Yes, No, Maybe) for the given question and provide a detailed explanation based on the context.",
    #     "Determine whether the answer is Yes, No, or Maybe and explain your choice using evidence from the context.",
    #     "Using the provided context, predict the correct answer and write a detailed response to justify it.",
    #     "Select the correct decision (Yes, No, Maybe) for the given question and elaborate on the reasoning.",
    #     "Provide both the correct answer and a comprehensive explanation to the given question using the context.",
    #     "Answer the question with Yes, No, or Maybe, and support your decision with details from the context.",
    #     "Generate a Yes, No, or Maybe answer for the question and add a long-form response explaining why.",
    #     "Decide the answer to the medical question (Yes, No, Maybe) and provide a detailed justification using the given context.",
    #     "Using the context, predict the answer (Yes, No, Maybe) and generate a detailed explanation for the question.",
    #     "Write the correct decision (Yes, No, Maybe) for the question and support it with relevant findings from the context."
    # ],
    # 'qa_short_answer': [
    #     "Choose the correct answer (Yes, No, or Maybe) for the given question based on the provided context.",
    #     "Determine the correct decision (Yes, No, or Maybe) for the medical question using the given context.",
    #     "From the provided information, select whether the answer to the question is Yes, No, or Maybe.",
    #     "Analyze the context and identify whether the question should be answered with Yes, No, or Maybe.",
    #     "Based on the given data, decide if the answer to the question is Yes, No, or Maybe."
    # ],
    # 'qa_long_answer': [
    #     "Generate a detailed answer to the question using the provided context.",
    #     "Create a comprehensive explanation for the question based on the given information.",
    #     "Write a long and detailed response to the question using the context provided.",
    #     "Using the context, provide an in-depth answer to the question with relevant details."
    # ],
    'summarization': [
        "Summarize the given article into a concise abstract that highlights the key findings and conclusions.",
        "Generate a brief and coherent abstract from the provided article text.",
        "Write a summary of the article that captures the main ideas and significant details.",
        "Condense the given article into a clear and concise abstract that represents its core content."
    ]
}

def assign_correct_instruction(row):
    task = row['task']
    if task in instruction_mapping:
        return random.choice(instruction_mapping[task])
    return "Instruction not found"

final_combined_train_sample['instruction'] = final_combined_train_sample.apply(assign_correct_instruction, axis=1)
final_combined_test_sample['instruction'] = final_combined_test_sample.apply(assign_correct_instruction, axis=1)

In [ ]:
import ast

def parse_features(row):
    try:
        return ast.literal_eval(row)
    except (ValueError, SyntaxError):
        return {}
        
def generate_input_output(row):
    input_value = ""; output_value = ""

    task = row['task']
    features = parse_features(row['features'])

    if row['dataset']=='bionli' and task=='nli':
        input_value = features.get('query', '').split('###')[1].replace("INPUT:", "").strip()
        output_value = features.get('answer', 'unknown')
    elif row['dataset']=='biorel' and task=='ie_extract_relation':
        input_value = features.get('text', '')
        output_value = features.get('relation', 'unknown')
    # elif row['dataset']=='biorel' and task=='ie_identify_relation_exist':
    #     input_value = features.get('text', '')
    #     output_value = 'YES' if features.get('relation') else 'NO'
    # elif row['dataset']=='biorel' and task=='ie_classify_relation':
    #     input_value = features.get('text', '')
    #     output_value = features.get('relation', 'unknown')
    # elif row['dataset']=='biorel' and task=='ie_generate_relation_pair':
    #     input_value = features.get('text', '')
    #     h_name = features.get('h', {}).get('name', 'unknown')
    #     t_name = features.get('t', {}).get('name', 'unknown')
    #     relation = features.get('relation', 'unknown')
    #     output_value = f"Entity Pair: ('{h_name}', '{t_name}'), Relationship: '{relation}'"
    elif row['dataset']=='icliniq' and task=='qa_subjective':
        input_value = features.get('input', '')
        output_value = features.get('answer_icliniq', '')
    elif row['dataset']=='medmcqa' and task=='qa_objective':
        question =  features.get('question', '').strip()
        options = {
            'A': features.get('opa', '').strip(),
            'B': features.get('opb', '').strip(),
            'C': features.get('opc', '').strip(),
            'D': features.get('opd', '').strip(),
        }

        cop_index = features.get('cop')
        if cop_index in [0, 1, 2, 3]:
            correct_option = ['A', 'B', 'C', 'D'][cop_index] 
            correct_answer = options[correct_option]
        else:
            correct_option = "Unknwon"
            correct_answer = "Invalid cop value"

        explanation = (features.get('exp') or '').strip()

        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"The answer is {correct_option}) {correct_answer}. "
            f"Explanation: {explanation}"
        )
    elif row['dataset']=='medqa' and task=='qa_objective':
        question = features.get('question', '').strip()
        options = features.get('options', {})

        answer_idx = features.get('answer_idx', '').strip()
        correct_answer = options.get(answer_idx, "Unknwon")

        input_value =  (
            f"{question} Please select one of the following: "
            f"A) {options.get('A', 'N/A')}, B) {options.get('B', 'N/A')}, "
            f"C) {options.get('C', 'N/A')}, D) {options.get('D', 'N/A')}."
        )
        output_value = f"The answer is {answer_idx}) {correct_answer}."

    # elif row['dataset']=='pubmedqa' and task=='qa_mesh_tagging':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     meshes = features.get('context', {}).get('meshes', [])
        
    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )
    #     output_value = f"MeSH Tags: {', '.join(meshes)}"
    # elif row['dataset']=='pubmedqa' and task=='qa_complex_answer':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     short_answer = features.get('final_decision', '').strip()  
    #     long_answer = features.get('long_answer', '').strip()  

    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )
    #     
    #     output_value = (
    #         f"Short Answer: {short_answer} "
    #         f"Long Answer: {long_answer}"
    #     )
    # elif row['dataset']=='pubmedqa' and task=='qa_short_answer':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     short_answer = features.get('long_answer', '').strip()  

    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )
    #     output_value = f"Answer: {short_answer}"

    # elif row['dataset']=='pubmedqa' and task=='qa_long_answer':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     long_answer = features.get('long_answer', '').strip() 

    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )

    #     output_value = f"Answer: {long_answer}"
    elif row['dataset']=='pubmed' and task=='summarization':
        features = ast.literal_eval(row['features'])  
        article = features.get('article', '').strip()  
        abstract = features.get('abstract', '').strip() 
        
        # Input 구성
        input_value = f"Summarize the following article: {article}"
        
        # Output 구성
        output_value = abstract
    return input_value, output_value

final_combined_train_sample[['input', 'output']] = final_combined_train_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

final_combined_test_sample[['input', 'output']] = final_combined_test_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

In [166]:
final_combined_train_sample.head(2)

,Unnamed: 0,dataset,split_data,features,input,output,task,instruction
0,212958,bionli,train,"{'id': 'BioNLI21940405', 'query': '\nTASK: Ple...",[PRE] Obesity and age are risk factors for fel...,entailment,nli,Read the given premise and hypothesis. Decide ...
1,214960,bionli,train,"{'id': 'BioNLI28524086', 'query': '\nTASK: Ple...",[PRE] Obesity is a risk factor for developing ...,entailment,nli,"Given a premise and a hypothesis, determine th..."


In [167]:
import pandas as pd

final_df = pd.concat([final_combined_train_sample, final_combined_test_sample])

In [168]:
final_df = final_df[~final_df['output'].str.contains('The answer is Unknwon)', regex=False)]

In [ ]:
final_combined_train_sample.to_json("src/data/data1_diabetes/train_instruction_dataset.json", orient="columns", indent=4)
final_combined_test_sample.to_json("src/data/data1_diabetes/test_instruction_dataset.json", orient="columns", indent=4)

: 